In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import spyrit.core.torch as st
from scipy.linalg import hadamard

**First Algorithm: Clients-to-server, aim to compute $B^{*}H^{*}D^{*}X^{*}$ and $B^{*}H^{*}D^{*}Y^{*}$**

In [2]:
## Function for randomized hadamard transform (rht)
def rht(X, Y):
# Generate a diagonal matrix of -1 or 1 with 0.5 probability
    diag_matrix = np.diag(np.random.choice([-1, 1], size=X.shape[0]))
    signs = diag_matrix
    # Multiply each row of X and Y by the corresponding sign
    result_X = np.dot(signs, X)
    result_Y = np.dot(signs, Y)
    # Convert to PyTorch tensor with float64
    tensor_matrix_X = torch.tensor(result_X, dtype=torch.float64)
    tensor_matrix_Y = torch.tensor(result_Y, dtype=torch.float64)
    # Apply FWHT
    tensor_matrix_X = st.fwht(tensor_matrix_X.T).T
    tensor_matrix_Y = st.fwht(tensor_matrix_Y)
    # Convert back to numpy array
    X = tensor_matrix_X.numpy() / np.sqrt(X.shape[0])
    Y = tensor_matrix_Y.numpy() / np.sqrt(Y.shape[0])
    return X, Y

In [3]:
def algorithm_for_BHDdot(n,k,p,weight,mu_1,mu_2,var_1,var_2,proportion,random_seed=39):
    m = n/k
    ## 2: Generate our GMM raw data, this (i) is not considered as the part of this scheme algorithm.
    mu_1_vector = np.ones(p) * mu_1
    sigma_1_vector = np.eye(p) * var_1
    mu_2_vector = np.ones(p) * mu_2
    sigma_2_vector = np.eye(p) * var_2
    weight_1 = weight
    data_proportion_1_no_rows = int(n * weight_1)
    data_proportion_2_no_rows = n - data_proportion_1_no_rows
    np.random.seed(random_seed)
    data_1 = np.random.multivariate_normal(mu_1_vector, sigma_1_vector,size=data_proportion_1_no_rows)
    data_2 = np.random.multivariate_normal(mu_2_vector, sigma_2_vector,size=data_proportion_2_no_rows)
    X_raw = np.vstack((data_1, data_2))
    np.random.shuffle(X_raw)
    epsilon = np.random.normal(0,1,size = n)
    beta = np.random.normal(0,1,size = p)
    Y_raw = np.dot(X_raw,beta) + epsilon
    ## 3: Use the np.random.shuffle indice method to partition the data equally to k clients.
    indices = np.random.permutation(n)
    X_shuffled = X_raw[indices]
    Y_shuffled = Y_raw[indices]
    X_clients = np.array_split(X_shuffled, k)
    Y_clients = np.array_split(Y_shuffled, k)
    ## 4: Apply normalized rht on each client side
    X_rht_clients = []
    Y_rht_clients = []
    for i in range(k):
        X_rht, Y_rht = rht(X_clients[i], Y_clients[i].reshape(-1, 1))
        X_rht_clients.append(X_rht)
        Y_rht_clients.append(Y_rht)
    ## 5a: Generate a randomized np.index list, the length is 'proportion' * m for each client (but fixed for all of them_meaning the same indices are used for all clients), index range is from 0 to m-1 (inclusive).
    np.random.seed(random_seed)
    selected_indices = np.random.choice(m, size=int(proportion * m), replace=False)
    ## 5b: make this index list to a diagonal matrix, with 1 on the selected indices and 0 elsewhere, so the dimension should be m x m.
    S_matrix = np.zeros((m, m))
    for index in selected_indices:
        S_matrix[index, index] = 1
    ## 6: Use this index matrix S to do on each client: S*H_m*D_i*X_i and S*H_m*D_i*Y_i (this process could be done either by using 5a or 5b, choose one which is faster).
    X_final_clients = []
    Y_final_clients = []
    for i in range(k):
        X_final = np.dot(S_matrix, X_rht_clients[i])
        Y_final = np.dot(S_matrix, Y_rht_clients[i])
        X_final_clients.append(X_final)
        Y_final_clients.append(Y_final)
    ## 7a: concatenate all the clients' final X and Y by columns to recover the dimension of (proportion*n) x p for X and (proportion*n) x 1 for Y.
    X_final_all = np.vstack(X_final_clients)
    Y_final_all = np.vstack(Y_final_clients)
    ## 7b: Compute the kronecker product: H_k ⊗ I_m first (H_k is the normalized hadamard matrix with dimension k xk, I_m is the identity matrix with dimension m x m), then multiply it with the concatenated X and Y from 7a.
    H_k = hadamard(k).astype(np.float64) / np.sqrt(k, dtype=np.float64)
    I_m = np.eye(m, dtype=np.float64)
    H_k_kron_I_m = np.kron(H_k, I_m)
    X_global = np.dot(H_k_kron_I_m, X_final_all)
    Y_global = np.dot(H_k_kron_I_m, Y_final_all)
    return X_global, Y_global